# Bi-linear interaction model with group-by augmentations

## Prepare dataset for training

Following the same steps as in [the training of a simple model](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb), we first load splitted dataset generated in [notebook](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/dataset_preprocessing/movielens%20with%20imdb.ipynb)

In [1]:
DATASET = 'rees_ecommerce'

In [2]:
import tensorflow as tf

from utils import load_dataset

datasets = {}
for split_name in ['train', 'val', 'test']:
    datasets[split_name] = load_dataset(DATASET, split_name)

2023-03-05 22:04:22.130831: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 22:04:22.315242: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2023-03-05 22:04:22.315283: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-05 22:04:23.539241: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


2023-03-05 22:04:25.410985: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2023-03-05 22:04:25.411043: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-05 22:04:25.411088: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (d0c4fb6a256c): /proc/driver/nvidia/version does not exist
2023-03-05 22:04:25.411358: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other 

Then we parse features' names to obtain a list of offer features (that will be used to modelize film) and a list of user features (aggregated history up to chosen date)

In [3]:
from utils import AGG_PREFIX

all_columns = list(datasets['train'].element_spec.keys())
technical_columns = ['user_id', 'date']
user_features = list(filter(lambda x: x.startswith(AGG_PREFIX), all_columns))
offer_features = list(filter(lambda x: x not in user_features + technical_columns, all_columns))

### Rebatching datasets

Splitting dataset into smaller batches in the same way as described in [the training of a simple model](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb)

In [4]:
%%time

from functools import partial
from uuid import uuid4

from utils import rebatch_by_events

datasets['train'] = rebatch_by_events(datasets['train'], batch_size=5040, date_column='date', nb_events_by_user_by_day=8)
for key in ['val', 'test']:
    datasets[key] = rebatch_by_events(datasets[key], batch_size=5040, date_column='date', nb_events_by_user_by_day=8,
                                      seed=1729).cache(f'/tmp/{uuid4()}.tf')

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
CPU times: user 27 s, sys: 1.11 s, total: 28.1 s
Wall time: 12.9 s


In [5]:
from utils import add_equal_weights

for key in datasets:
    datasets[key] = datasets[key].map(partial(add_equal_weights, features=offer_features))

## Define the model

First we need to get number of different modalities inputs can take from saved vectorizers (it will be used in embeddings layer definition):

In [6]:
from utils import load_inverse_lookups
inverse_lookups = load_inverse_lookups(DATASET)

In [7]:
import re

vocabulary_sizes = {}

for feature in offer_features:
    vocabulary_sizes[feature] = inverse_lookups[feature].vocabulary_size()

for feature in user_features:
    for key in inverse_lookups:
        pattern = re.compile(r"{}(\w+)_{}".format(AGG_PREFIX, key))
        if pattern.match(feature):
            vocabulary_sizes[feature] = vocabulary_sizes[key]

### Model architecture

Now we can assemble all these layers into final model. Note that offer compression weights and interaction kernels are shared between different augmentations we generate.

<img src="resources/group_by_augmentations_model.png" alt="model" width="800" />

### Model parameters

In [ ]:
EMBEDDING_DIM = 100
L1_COEFF = 4e-7
DROPOUT = 0.1
NB_AUGMENTATIONS = 3
AVERAGE_NUMBER_OF_FEATURES_IN_AUGMENTATION = 2
USER_META_FEATURES = 5
OFFER_META_FEATURES = 3

def REGULARIZER():
    return {'class_name': 'L1L2', 'config': {'l1': L1_COEFF, 'l2': 0.}}

def OUTPUT_DNN():
    return tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(100,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(50,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(1,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
    ], name='output_Dnn')

EPOCHS = 10

NUMBER_OF_NEGATIVES = 4
LOSS = tf.keras.losses.BinaryCrossentropy(from_logits=True)
AUC_METRIC = tf.keras.metrics.AUC(from_logits=True)

import tensorflow_addons as tfa
OPTIMIZER = tfa.optimizers.AdamW(weight_decay=4e-8, learning_rate=0.0009)

### Embeddings

We will define embeddings with the same `WeightedEmbeddings` layer described in [the training of a simple model](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb) with one addition:
* for offer features when aggregating a list of embedding vectors, we will also calculate variance and not only mean vector

It is easy to do in the same sparse-dense matrix multiplication operation as mean calculation (we get second moment and then calculate variance from it).

In [26]:
from layers import get_input_layer, WeightedEmbeddings
from utils import WEIGHT_SUFFIX

inputs = {}
embedded_user_features, embedded_offer_features, variance_offer_features = {}, {}, {}
for feature in user_features:
    inputs[feature] = get_input_layer(feature)
    emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                   EMBEDDING_DIM, name=f'{feature}_embedding',
                                   embeddings_regularizer=REGULARIZER())
    embedded_user_features[feature] = emb_layer(inputs[feature])
for feature in offer_features:
    # for offer features we need weights:
    # with dummy weights during training, and the ones used for a feature's averaging at inference time
    inputs[f'{feature}_weight'] = get_input_layer(f'{feature}_weight', tf.float32)
    inputs[feature] = get_input_layer(feature)
    emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                   EMBEDDING_DIM, name=f'{feature}_embedding',
                                   embeddings_regularizer=REGULARIZER(),
                                   calculate_variance=True)
    embedded_offer_features[feature], variance_offer_features[feature] =\
        emb_layer(inputs[feature], inputs[f'{feature}_weight'])

### Combining everything into model

Now we can define described model architecture on the top of embeddings.

In [27]:
user_stacked = tf.stack(list(embedded_user_features.values()), axis=1)
offer_stacked = tf.stack(list(embedded_offer_features.values()), axis=1)
offer_variance = tf.stack(list(variance_offer_features.values()), axis=1)
stacked_raw_offer_attrs = tf.stack([tf.cast(inp.values, tf.int32) for feature, inp in inputs.items()
                                    if feature in offer_features], axis=1)

In [28]:
from layers import KeyGenerator, GroupBy
from layers import UserFeaturesCompressor
from layers import OfferFeaturesCompressor

from layers import MaskNet

from layers import BiLinearInteraction


group_by = GroupBy(name='group_by')
key_generator = KeyGenerator(number_of_offer_attributes=len(offer_features),
                             average_number_of_attributes_in_key=AVERAGE_NUMBER_OF_FEATURES_IN_AUGMENTATION,
                             name='grp_key_generator')

user_compressed = UserFeaturesCompressor(USER_META_FEATURES, DROPOUT,
                                         name='user_compressor')(user_stacked)
offer_features_compressor = OfferFeaturesCompressor(OFFER_META_FEATURES, DROPOUT, name='offer_compressor')
mask_net = MaskNet(OFFER_META_FEATURES, DROPOUT, name='mask_generation')
apply_mask = tf.keras.layers.Multiply(name='apply_mask')
bi_linear_interaction = BiLinearInteraction(number_of_negatives=NUMBER_OF_NEGATIVES, dropout_rate=DROPOUT,
                                            initializer='random_normal', regularizer=REGULARIZER(),
                                            name='interaction')
output_dnn = OUTPUT_DNN()

augmentation_predictions = []
for i in range(NB_AUGMENTATIONS):
    group_by_key = key_generator(stacked_raw_offer_attrs)
    mean_offer_emb, variance_offer_emb = group_by(group_by_key, offer_stacked)
    compressed_offer_embeddings = offer_features_compressor([mean_offer_emb, variance_offer_emb])
    mask = mask_net([mean_offer_emb, variance_offer_emb])
    masked_offer_embeddings = apply_mask([compressed_offer_embeddings, mask])
    _output = output_dnn(bi_linear_interaction([user_compressed, masked_offer_embeddings], generate_negatives=True))
    augmentation_predictions.append(_output)
output = tf.concat(augmentation_predictions, axis=1)

/opt/conda/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


And for evaluation we don't need to create augmentations, we need just to take offer features' mean and variance coming from inputs.

In [29]:
compressed_offer_embeddings = offer_features_compressor([offer_stacked, offer_variance])
mask = mask_net([offer_stacked, offer_variance])
masked_offer_embeddings = apply_mask([compressed_offer_embeddings, mask])

eval_output = output_dnn(bi_linear_interaction([user_compressed, masked_offer_embeddings], generate_negatives=True))

In [30]:
from utils import BroadcastLoss, BroadcastMetric

model = tf.keras.Model(inputs, output, name='group_by_augmentations')
model.compile(optimizer=OPTIMIZER,
              loss=BroadcastLoss(LOSS, NUMBER_OF_NEGATIVES),
              metrics=[BroadcastMetric(AUC_METRIC, NUMBER_OF_NEGATIVES)])

eval_model = tf.keras.Model(inputs, eval_output, name='group_by_augmentations_eval')

### Training

In [14]:
model.fit(datasets['train'], epochs=EPOCHS, validation_data=datasets['val'])

Epoch 1/10


/opt/conda/lib/python3.7/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'date'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


    682/Unknown - 647s 859ms/step - loss: 0.4643 - auc: 0.7114

2023-03-05 22:17:48.991534: W tensorflow/core/kernels/data/cache_dataset_ops.cc:296] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


682/682 [==============================] - 709s 949ms/step - loss: 0.4643 - auc: 0.7114 - val_loss: 0.4425 - val_auc: 0.7324


2023-03-05 22:18:47.881811: W tensorflow/core/kernels/data/cache_dataset_ops.cc:296] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Epoch 2/10
682/682 [==============================] - 608s 891ms/step - loss: 0.4392 - auc: 0.7361 - val_loss: 0.4379 - val_auc: 0.7401
Epoch 3/10
682/682 [==============================] - 590s 864ms/step - loss: 0.4352 - auc: 0.7423 - val_loss: 0.4387 - val_auc: 0.7383
Epoch 4/10
682/682 [==============================] - 584s 856ms/step - loss: 0.4343 - auc: 0.7439 - val_loss: 0.4373 - val_auc: 0.7402
Epoch 5/10
682/682 [==============================] - 587s 860ms/step - loss: 0.4335 - auc: 0.7453 - val_loss: 0.4364 - val_auc: 0.7421
Epoch 6/10
682/682 [==============================] - 579s 848ms/step - loss: 0.4325 - auc: 0.7471 - val_loss: 0.4377 - val_auc: 0.7411
Epoch 7/10
682/682 [==============================] - 582s 853ms/step - loss: 0.4319 - auc: 0.7482 - val_loss: 0.4332 - val_auc: 0.7463
Epoch 8/10
682/682 [==============================] - 583s 855ms/step - loss: 0.4316 - auc: 0.7491 - val_loss: 0.4374 - val_auc: 0.7411
Epoch 9/10
682/682 [============================

## Single task models benchmark

Using same approach as in [the simple model notebook](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb) we will look at performance gap between the model with group by augmentations against set of models specialized on tasks corresponding to one offer feature at time. We won't use augmentations in those baseline models, because they will be already aligned with offer we will use in evaluation afterwards.

In [15]:
# offer columns we want to evaluate, specific to dataset we test
TASKS = ['product_id', 'category1', 'category2', 'category3', 'brand', 'priceCluster']

In [16]:
def bi_linear_interaction_model(single_task_feature, name='bi_linear_model'):
    # user_features, vocabulary_sizes, EMBEDDING_DIM, REGULARIZER, USER_TOWER, OFFER_TOWER,
    # OPTIMIZER, LOSS, NUMBER_OF_NEGATIVES
    # come from global scope, but can be passed as params instead
    inputs = {}
    embedded_user_features, embedded_offer_features, variance_offer_features = {}, {}, {}
    for feature in user_features:
        inputs[feature] = get_input_layer(feature)
        emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                       EMBEDDING_DIM, name=f'{feature}_embedding',
                                       embeddings_regularizer=REGULARIZER())
        embedded_user_features[feature] = emb_layer(inputs[feature])

    # for offer feature we need weights:
    # with dummy weights during training, and the ones used for a feature's averaging at inference time
    inputs[f'{single_task_feature}_weight'] = get_input_layer(f'{single_task_feature}_weight', tf.float32)
    inputs[single_task_feature] = get_input_layer(single_task_feature)
    emb_layer = WeightedEmbeddings(vocabulary_sizes[single_task_feature],
                                   EMBEDDING_DIM, name=f'{single_task_feature}_embedding',
                                   embeddings_regularizer=REGULARIZER())
    embedded_offer_feature = emb_layer(inputs[single_task_feature],
                                       inputs[f'{single_task_feature}_weight'])
    
    user_stacked = tf.stack(list(embedded_user_features.values()), axis=1)
    offer_stacked = tf.expand_dims(embedded_offer_feature, axis=1)
    
    
    user_compressed = UserFeaturesCompressor(USER_META_FEATURES, DROPOUT,
                                             name='user_compressor')(user_stacked)
    mask_net = MaskNet(OFFER_META_FEATURES, DROPOUT, name='mask_generation')
    apply_mask = tf.keras.layers.Multiply(name='apply_mask')
    bi_linear_interaction = BiLinearInteraction(number_of_negatives=NUMBER_OF_NEGATIVES, dropout_rate=DROPOUT,
                                                initializer='random_normal', regularizer=REGULARIZER(),
                                                name='interaction')
    output_dnn = OUTPUT_DNN()

    
    mask = mask_net([offer_stacked, offer_stacked])
    masked_offer_embeddings = apply_mask([offer_stacked, mask])
    
    output = OUTPUT_DNN()(bi_linear_interaction([user_compressed, masked_offer_embeddings],
                                                generate_negatives=True))

    model = tf.keras.Model(inputs, output, name=name)
    model.compile(optimizer=OPTIMIZER,
                  loss=BroadcastLoss(LOSS, NUMBER_OF_NEGATIVES),
                  metrics=[BroadcastMetric(AUC_METRIC, NUMBER_OF_NEGATIVES)])
    
    return model

In [ ]:
mono_feature_models = {}
for task_offer_feature in TASKS:
    mono_feature_models[task_offer_feature] =\
        bi_linear_interaction_model(task_offer_feature, name=f'{task_offer_feature}_model')
    mono_feature_models[task_offer_feature].fit(datasets['train'],
                                                epochs=EPOCHS,
                                                validation_data=datasets['val'])

Epoch 1/10


/opt/conda/lib/python3.7/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'category2', 'category1', 'brand', 'category3', 'priceCluster', 'date', 'category2_weight', 'category1_weight', 'brand_weight', 'category3_weight', 'priceCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


682/682 [==============================] - 331s 439ms/step - loss: 0.4645 - auc: 0.7330 - val_loss: 0.4477 - val_auc: 0.7521
Epoch 2/10
682/682 [==============================] - 289s 423ms/step - loss: 0.4397 - auc: 0.7602 - val_loss: 0.4407 - val_auc: 0.7577
Epoch 3/10
682/682 [==============================] - 288s 422ms/step - loss: 0.4327 - auc: 0.7685 - val_loss: 0.4396 - val_auc: 0.7592
Epoch 4/10
682/682 [==============================] - 284s 416ms/step - loss: 0.4292 - auc: 0.7736 - val_loss: 0.4410 - val_auc: 0.7596
Epoch 5/10
682/682 [==============================] - 293s 429ms/step - loss: 0.4267 - auc: 0.7776 - val_loss: 0.4403 - val_auc: 0.7602
Epoch 6/10
682/682 [==============================] - 292s 428ms/step - loss: 0.4251 - auc: 0.7809 - val_loss: 0.4413 - val_auc: 0.7599
Epoch 7/10
682/682 [==============================] - 289s 423ms/step - loss: 0.4238 - auc: 0.7839 - val_loss: 0.4467 - val_auc: 0.7593
Epoch 8/10
682/682 [==============================] - 290s 

/opt/conda/lib/python3.7/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'category2', 'product_id', 'brand', 'category3', 'priceCluster', 'date', 'category2_weight', 'product_id_weight', 'brand_weight', 'category3_weight', 'priceCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


682/682 [==============================] - 337s 449ms/step - loss: 0.4789 - auc: 0.6784 - val_loss: 0.4716 - val_auc: 0.6728
Epoch 2/10
682/682 [==============================] - 288s 422ms/step - loss: 0.4714 - auc: 0.6721 - val_loss: 0.4707 - val_auc: 0.6739
Epoch 3/10
682/682 [==============================] - 282s 413ms/step - loss: 0.4701 - auc: 0.6734 - val_loss: 0.4696 - val_auc: 0.6743
Epoch 4/10
682/682 [==============================] - 278s 408ms/step - loss: 0.4693 - auc: 0.6744 - val_loss: 0.4692 - val_auc: 0.6744
Epoch 5/10
682/682 [==============================] - 289s 423ms/step - loss: 0.4688 - auc: 0.6753 - val_loss: 0.4692 - val_auc: 0.6745
Epoch 6/10
682/682 [==============================] - 291s 426ms/step - loss: 0.4683 - auc: 0.6763 - val_loss: 0.4691 - val_auc: 0.6748
Epoch 7/10
682/682 [==============================] - 284s 416ms/step - loss: 0.4679 - auc: 0.6773 - val_loss: 0.4695 - val_auc: 0.6739
Epoch 8/10
682/682 [==============================] - 287s 

...

682/682 [==============================] - 338s 444ms/step - loss: 0.4722 - auc: 0.6768 - val_loss: 0.4638 - val_auc: 0.6877
Epoch 2/10
682/682 [==============================] - 290s 425ms/step - loss: 0.4629 - auc: 0.6873 - val_loss: 0.4621 - val_auc: 0.6891
Epoch 3/10
682/682 [==============================] - 286s 419ms/step - loss: 0.4614 - auc: 0.6890 - val_loss: 0.4617 - val_auc: 0.6894
Epoch 4/10
682/682 [==============================] - 280s 410ms/step - loss: 0.4605 - auc: 0.6903 - val_loss: 0.4610 - val_auc: 0.6899
Epoch 5/10
682/682 [==============================] - 285s 417ms/step - loss: 0.4600 - auc: 0.6912 - val_loss: 0.4612 - val_auc: 0.6901
Epoch 6/10
682/682 [==============================] - 280s 410ms/step - loss: 0.4594 - auc: 0.6925 - val_loss: 0.4613 - val_auc: 0.6897
Epoch 7/10
682/682 [==============================] - 279s 409ms/step - loss: 0.4590 - auc: 0.6935 - val_loss: 0.4616 - val_auc: 0.6899
Epoch 8/10
682/682 [==============================] - 288s 

/opt/conda/lib/python3.7/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'category2', 'product_id', 'category1', 'brand', 'priceCluster', 'date', 'category2_weight', 'product_id_weight', 'category1_weight', 'brand_weight', 'priceCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


682/682 [==============================] - 347s 458ms/step - loss: 0.4742 - auc: 0.6732 - val_loss: 0.4655 - val_auc: 0.6816
Epoch 2/10
682/682 [==============================] - 290s 424ms/step - loss: 0.4645 - auc: 0.6816 - val_loss: 0.4638 - val_auc: 0.6834
Epoch 3/10
682/682 [==============================] - 283s 414ms/step - loss: 0.4629 - auc: 0.6836 - val_loss: 0.4632 - val_auc: 0.6843
Epoch 4/10
682/682 [==============================] - 296s 434ms/step - loss: 0.4620 - auc: 0.6849 - val_loss: 0.4630 - val_auc: 0.6828
Epoch 5/10
682/682 [==============================] - 293s 429ms/step - loss: 0.4614 - auc: 0.6861 - val_loss: 0.4629 - val_auc: 0.6848
Epoch 6/10
682/682 [==============================] - 304s 445ms/step - loss: 0.4611 - auc: 0.6869 - val_loss: 0.4628 - val_auc: 0.6850
Epoch 7/10
682/682 [==============================] - 308s 452ms/step - loss: 0.4607 - auc: 0.6880 - val_loss: 0.4625 - val_auc: 0.6847
Epoch 8/10
682/682 [==============================] - 296s 

/opt/conda/lib/python3.7/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['user_id', 'category2', 'product_id', 'category1', 'category3', 'priceCluster', 'date', 'category2_weight', 'product_id_weight', 'category1_weight', 'category3_weight', 'priceCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


682/682 [==============================] - 338s 446ms/step - loss: 0.4584 - auc: 0.7127 - val_loss: 0.4465 - val_auc: 0.7316
Epoch 2/10
682/682 [==============================] - 290s 424ms/step - loss: 0.4445 - auc: 0.7331 - val_loss: 0.4432 - val_auc: 0.7347
Epoch 3/10
682/682 [==============================] - 285s 418ms/step - loss: 0.4418 - auc: 0.7364 - val_loss: 0.4421 - val_auc: 0.7360
Epoch 4/10
682/682 [==============================] - 287s 419ms/step - loss: 0.4404 - auc: 0.7383 - val_loss: 0.4415 - val_auc: 0.7359
Epoch 5/10
682/682 [==============================] - 289s 424ms/step - loss: 0.4394 - auc: 0.7398 - val_loss: 0.4413 - val_auc: 0.7363
Epoch 6/10
682/682 [==============================] - 290s 425ms/step - loss: 0.4386 - auc: 0.7412 - val_loss: 0.4414 - val_auc: 0.7371
Epoch 7/10
682/682 [==============================] - 290s 425ms/step - loss: 0.4381 - auc: 0.7424 - val_loss: 0.4416 - val_auc: 0.7371
Epoch 8/10
682/682 [==============================] - 287s 

## Evaluation

In [ ]:
%%time
from utils import prepare_single_task_dataset
test_datasets = {}
for task_offer_feature in TASKS:
    test_datasets[task_offer_feature] = \
        prepare_single_task_dataset(datasets['test'], task_offer_feature, offer_features)

In [ ]:
%%time
from collections import defaultdict
from utils import evaluate_model, wAUC

aucs = defaultdict(dict)
for task_offer_feature in TASKS:
    for model_name in TASKS:
        aucs[task_offer_feature][f'MONO:{model_name}'] = \
            evaluate_model(mono_feature_models[model_name],
                           task_offer_feature, test_datasets, inverse_lookups, NUMBER_OF_NEGATIVES)
    aucs[task_offer_feature]['new model'] = \
            evaluate_model(eval_model, task_offer_feature, test_datasets, inverse_lookups, NUMBER_OF_NEGATIVES)

In [ ]:
import pandas as pd
results = pd.DataFrame()
for task_name in aucs:
    for model_name in aucs[task_name]:
        w_auc = wAUC(aucs[task_name][model_name])
        results = results.append({'wAUC': w_auc, 'offers': task_name, 'model': model_name}, ignore_index=True)

In [24]:
pd.pivot_table(results, 'wAUC', 'model', 'offers').style.background_gradient(cmap='coolwarm')

offers,brand,category1,category2,category3,priceCluster,product_id
model,,,,,,
MONO:brand,0.729320,0.584560,0.602323,0.592103,0.601051,0.730183
MONO:category1,0.622370,0.665993,0.662037,0.639652,0.568841,0.657654
MONO:category2,0.625577,0.645309,0.680136,0.648451,0.567752,0.662823
MONO:category3,0.627817,0.627434,0.645139,0.675197,0.566485,0.657189
MONO:priceCluster,0.642769,0.571176,0.580199,0.584151,0.699928,0.721428
MONO:product_id,0.700882,0.619597,0.625432,0.639249,0.648758,0.766634
new model,0.714909,0.652915,0.651079,0.645676,0.672518,0.766951
